In [9]:
import cascade as csc
import numpy as np
import pykep as pk
from copy import deepcopy
from tqdm.notebook import tqdm

In [10]:
nparts = 17378

In [4]:
#csc.set_logger_level_trace()

In [5]:
rng = np.random.default_rng()

x = []
y = []
z = []

vx = []
vy = []
vz = []

sizes = []

for _ in range(nparts):
    a = rng.uniform(1.02, 1.3)
    e = rng.uniform(0, 0.02)
    inc = rng.uniform(0, 0.05)
    om = rng.uniform(0, 2*np.pi)
    Om = rng.uniform(0, 2*np.pi)
    nu = rng.uniform(0, 2*np.pi)
    size = rng.uniform(1.57e-8, 1.57e-7)
    
    r, v = pk.par2ic([a, e, inc, om, Om, nu])
    
    x.append(r[0])
    y.append(r[1])
    z.append(r[2])

    vx.append(v[0])
    vy.append(v[1])
    vz.append(v[2])
    
    sizes.append(size)

sim = csc.sim(x,y,z,vx,vy,vz,sizes,0.23)

[2022-01-26 15:06:15.853] [cascade] [info] cascade logger initialised


In [6]:
final_t = 86400*365/806.81

In [ ]:
pbar = tqdm(total=final_t)

while sim.time < final_t:
    orig_time = sim.time
    
    oc = sim.step()
    
    pbar.update(sim.time - orig_time)
   
    if oc == csc.outcome.collision:
        # TODO different code needed for crash
        # on Earth here.
        pi, pj = sim.interrupt_info
        
        print("Collision detected, re-initing particles {} and {}".format(pi, pj))
        
        new_x = deepcopy(sim.x)
        new_y = deepcopy(sim.y)
        new_z = deepcopy(sim.z)
        
        new_vx = deepcopy(sim.vx)
        new_vy = deepcopy(sim.vy)
        new_vz = deepcopy(sim.vz)
        
        new_sizes = deepcopy(sim.sizes)

        for idx in [pi, pj]:
            a = rng.uniform(1.02, 1.3)
            e = rng.uniform(0, 0.02)
            inc = rng.uniform(0, 0.05)
            om = rng.uniform(0, 2*np.pi)
            Om = rng.uniform(0, 2*np.pi)
            nu = rng.uniform(0, 2*np.pi)
            size = rng.uniform(1.57e-8, 1.57e-7)

            r, v = pk.par2ic([a, e, inc, om, Om, nu])

            new_x[idx] = r[0]
            new_y[idx] = r[1]
            new_z[idx] = r[2]

            new_vx[idx] = v[0]
            new_vy[idx] = v[1]
            new_vz[idx] = v[2]
            
            new_sizes[idx] = size

        sim.set_new_state(new_x, new_y, new_z, new_vx, new_vy, new_vz, new_sizes)

pbar.close()
del pbar